In [1]:
import numpy as np
import tensorflow as tf
import pickle

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def scale_img(image):
    return_img = np.zeros_like(image, dtype = float)
    
    r_img = image[:,0:1024]
    r_avg = np.average(r_img)
    r_std = np.std(r_img)
    r_img = (r_img - r_avg)/r_std
    return_img[:,0:1024] = r_img

    g_img = image[:,1024:2048]
    g_avg = np.average(g_img)
    g_std = np.std(g_img)
    g_img = (g_img - g_avg)/g_std
    return_img[:,1024:2048] = g_img

    b_img = image[:,2048:]
    b_avg = np.average(b_img)
    b_std = np.std(b_img)
    b_img = (b_img - b_avg)/b_std
    return_img[:,2048:] = b_img
    
    #return ((image/255.)-0.5)
    return return_img

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

def unpickle_all(path):
    train_images = np.zeros((50000,32,32,3))
    train_stack = np.zeros((50000,3072))
    test_images = np.zeros((10000,32,32,3))
    train_labels = np.zeros((50000))
    test_labels = np.zeros((10000))
    for i in range(1,6):
        j = i-1
        cifar10 = unpickle(path+'/data_batch_{}'.format(i))
        train_labels[j*10000:(j+1)*10000] = cifar10['labels']
        #data = scale_img(cifar10['data'])
        #train_images[j*10000:(j+1)*10000,:,:,:] = np.swapaxes(np.reshape(data,(10000,32,32,3), order='F'),axis1=1,axis2=2)
        train_stack[j*10000:(j+1)*10000,:] = cifar10['data']
    
    train_stack = scale_img(train_stack)
    train_images = np.swapaxes(np.reshape(train_stack,(50000,32,32,3), order='F'),axis1=1,axis2=2)
    
    cifar10 = unpickle(path + '/test_batch')
    test_labels = cifar10['labels']
    data = scale_img(cifar10['data'])
    test_images = np.swapaxes(np.reshape(data,(10000,32,32,3), order='F'),axis1=1,axis2=2)

    return train_images, train_labels, test_images, test_labels

In [3]:
images, labels, test_images, test_labels = unpickle_all('./storage/CIFAR-10')

In [4]:
tf.reset_default_graph()
momentum = 0.9
lambd = 0.0005
epoch = 200
batch_size = 128
display_step = 2
total_batch = int(images.shape[0] / batch_size)

global_step = tf.Variable(0, trainable=False)
boundaries = [100*total_batch-1, 150*total_batch]
values = [0.001, 0.0005, 0.0001]
LR = tf.train.piecewise_constant(global_step, boundaries, values)

In [5]:
x = tf.placeholder('float32', [None,32,32,3], name = "X")
y = tf.placeholder('int64', [None,], name = "Y")

#OUT_CL_1 : n x 14 x 14 x 6
#OUT_CL_2 : n x 5 x 5 x 16 = n x 400
#OUT_DENSE_1 : n x 120
#OUT_DENSE_2 : n x 84

W_CL_1 = tf.Variable(np.random.randn(5, 5, 3, 6)/25, dtype="float32", name = "W_CL_1")
b_CL_1 = tf.Variable(np.random.randn(1, 1, 1, 6), dtype="float32", name = "b_CL_1")
W_CL_2 = tf.Variable(np.random.randn(5, 5, 6, 16)/25, dtype="float32", name = "W_CL_2")
b_CL_2 = tf.Variable(np.random.randn(1, 1, 1, 16), dtype="float32", name = "b_CL_2")

REG_W_CL_1 = tf.square(tf.norm(W_CL_1))
REG_b_CL_1 = tf.square(tf.norm(b_CL_1))
REG_W_CL_2 = tf.square(tf.norm(W_CL_2))
REG_b_CL_2 = tf.square(tf.norm(b_CL_2))

REGUL = lambd*(REG_W_CL_1 + REG_b_CL_1 + REG_W_CL_2 + REG_b_CL_2)/(2 * batch_size)

with tf.name_scope("CL1"):
    CL_1 = tf.nn.conv2d(x,W_CL_1,strides = [1,1,1,1], padding="VALID", name = "CL_1")
    RELU_1 = tf.nn.relu(tf.add(CL_1,b_CL_1), name = "RELU_1")
    MAXPOOL_1 = tf.nn.max_pool(RELU_1,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID", name = "MAXPOOL_1")

with tf.name_scope("CL2"):
    CL_2 = tf.nn.conv2d(MAXPOOL_1,W_CL_2,strides = [1,1,1,1], padding="SAME", name = "CL_2")
    RELU_2 = tf.nn.relu(tf.add(CL_2,b_CL_2), name = "RELU_2")
    MAXPOOL_2 = tf.nn.max_pool(RELU_2,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID", name = "MAXPOOL_2")
    FLAT_2 = tf.layers.flatten(MAXPOOL_2, name = "FLAT_2")

with tf.name_scope("DENSE_1"):
    DENSE_1 = tf.contrib.layers.fully_connected(FLAT_2,120,weights_regularizer= tf.contrib.layers.l2_regularizer(lambd), biases_regularizer =tf.contrib.layers.l2_regularizer(lambd))

with tf.name_scope("DENSE_2"):
    DENSE_2 = tf.contrib.layers.fully_connected(DENSE_1,84,weights_regularizer= tf.contrib.layers.l2_regularizer(lambd), biases_regularizer =tf.contrib.layers.l2_regularizer(lambd))

with tf.name_scope("LOGITS_1"):
    LOGITS_1 = tf.contrib.layers.fully_connected(DENSE_2,10, activation_fn = None,weights_regularizer= tf.contrib.layers.l2_regularizer(lambd), biases_regularizer =tf.contrib.layers.l2_regularizer(lambd))

with tf.name_scope("PREDICTIONS"):
    y_hat = tf.nn.softmax(LOGITS_1, name = "y_hat")
    predictions = tf.argmax(y_hat,1)

with tf.name_scope("LOSS"):
    loss = tf.add(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y, depth=10),logits=LOGITS_1)),REGUL)
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y, depth=10),logits=LOGITS_1))
    tf.summary.scalar("cost_function", loss)

with tf.name_scope("ACCURACY"):
    compare = tf.equal(predictions, y)
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
    tf.summary.scalar("Accuracy", accuracy)

with tf.name_scope("OPTIMIZER"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate = LR, momentum = momentum, name="opt").minimize(loss, global_step = global_step)
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
    
init = tf.global_variables_initializer()
summ = tf.summary.merge_all()

In [6]:
with tf.Session() as sess:
    np.random.seed(1)
    sess.run(init)
    summary_writer = tf.summary.FileWriter('data/logs/2')
    summary_writer.add_graph(sess.graph)
    # Training cycle
    for iteration in range(epoch):
        avg_cost = 0.
        avg_train_accuracy = 0.
        total_batch = int(images.shape[0] / batch_size)
        # Loop over all batches
        total_batch_time = 0.

        for i in range(total_batch):
            batch_xs = images[i*batch_size:(i+1)*batch_size,:,:,:]
            batch_ys = labels[i*batch_size:(i+1)*batch_size]
            _, cost, train_accuracy, summary_str = sess.run([optimizer, loss, accuracy, summ], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += cost/batch_size/total_batch
            avg_train_accuracy += train_accuracy/total_batch
            summary_writer.add_summary(summary_str, iteration * total_batch + i)

        if iteration % display_step == 0:
            test_accuracy = sess.run([accuracy], feed_dict={x: test_images, y: test_labels})
            print("Iteration: {}  Cost= {}  Train accuracy= {}%  Test accuracy= {}%".format(iteration, avg_cost, avg_train_accuracy*100,test_accuracy[0]*100))

Iteration: 0  Cost= 0.031601909216111286  Train accuracy= 21.10176282051285%  Test accuracy= 26.930001378059387%
Iteration: 2  Cost= 0.013751865675052016  Train accuracy= 35.751201923077005%  Test accuracy= 36.86000108718872%
Iteration: 4  Cost= 0.012353908366117723  Train accuracy= 42.70633012820516%  Test accuracy= 40.529999136924744%
Iteration: 6  Cost= 0.011462590941347372  Train accuracy= 47.12339743589747%  Test accuracy= 45.03999948501587%
Iteration: 8  Cost= 0.01078812735489545  Train accuracy= 50.45472756410261%  Test accuracy= 47.17000126838684%
Iteration: 10  Cost= 0.01029966220689508  Train accuracy= 52.844551282051334%  Test accuracy= 48.69999885559082%
Iteration: 12  Cost= 0.009896101167377757  Train accuracy= 54.73758012820519%  Test accuracy= 49.77000057697296%
Iteration: 14  Cost= 0.009577940326804922  Train accuracy= 56.268028846153896%  Test accuracy= 50.41000247001648%
Iteration: 16  Cost= 0.009316392481709136  Train accuracy= 57.4859775641026%  Test accuracy= 51.82

Iteration: 146  Cost= 0.0034980216869511323  Train accuracy= 84.59134615384617%  Test accuracy= 51.340001821517944%
Iteration: 148  Cost= 0.003373323041850178  Train accuracy= 85.08012820512816%  Test accuracy= 51.30000114440918%
Iteration: 150  Cost= 0.003389651645631649  Train accuracy= 85.13822115384616%  Test accuracy= 50.849997997283936%
Iteration: 152  Cost= 0.003397538337426692  Train accuracy= 85.14222756410263%  Test accuracy= 50.7099986076355%
Iteration: 154  Cost= 0.0033475369752313066  Train accuracy= 85.34855769230778%  Test accuracy= 50.62999725341797%
Iteration: 156  Cost= 0.0033310298990004506  Train accuracy= 85.625%  Test accuracy= 50.679999589920044%
Iteration: 158  Cost= 0.0032754861224347184  Train accuracy= 85.69511217948725%  Test accuracy= 51.15000009536743%
Iteration: 160  Cost= 0.003274365069452101  Train accuracy= 85.75320512820514%  Test accuracy= 50.87000131607056%
Iteration: 162  Cost= 0.0032130571040444255  Train accuracy= 85.89342948717953%  Test accurac